In [1]:
# jmframe: I am making this script to loop through the nwm land surface files (LDAS)
# and combine all the years for each basin.
import copy
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import numpy as np
import os
import pandas as pd
import pickle
import sys
import time
import tools
from tqdm import tqdm

In [29]:
data_v2_dir = '/YOUR DIRECTORY HERE/v2/'
data_v2_dir_cam = data_v2_dir+'/YOUR DIRECTORY HERE/'
proj_data_dir = '/YOUR PROJECT DIRECTORY HERE/data/'
combine_years = False
average_days = False
start_end_year_list = [1993, 2018]
rest_of_years_list = [y for y in range(start_end_year_list[0]+1, start_end_year_list[1])]

In [3]:
# Here I am going to load all of the individual years with the collated data, 
# then put it together.
df = {}
# Load these as a samples no matter what, we want their start/end dates:
file_name_prefix = 'dynamic_features_nwm_RT_'
for y in start_end_year_list:
    with open(data_v2_dir_cam+file_name_prefix+str(y)+'.p', 'rb') as pf:
        df[y] = pickle.load(pf)

# Then the rest of the individual years, if needed
if combine_years:
    for y in rest_of_years_list:
        with open(data_v2_dir_cam+file_name_prefix+str(y)+'.p', 'rb') as pf:
            df[y] = pickle.load(pf)
else:
    print('RT years not loaded here')

In [4]:
basins = list(df[1993].keys())
sample_basin = basins[0]
firstdt = df[start_end_year_list[0]][sample_basin].index.values[0]
lastd = df[start_end_year_list[1]][sample_basin].index.values[-1]
date_listn= pd.date_range(start=firstdt, end=lastd, freq='3H')
feature_names = list(df[1993][sample_basin].columns.values)
n_dates = date_listn.shape[0]
n_features = len(feature_names)

In [5]:
# Set up the dictionary with a dataframe for each basin,
# then append in the year values from our files.
if combine_years:
    rt = {b:pd.DataFrame(columns=feature_names) for b in basins}
    for b in basins:
        for y in df.keys():
            rt[b] = rt[b].append(df[y][b])
        rt[b] = rt[b].sort_index()
else:
    print('LDAS years not combined here')

In [30]:
# Now average the basins by day, instead of 3 hour intervals.
if average_days:
    def daily_average(df):
        return(df.groupby(pd.Grouper(freq='1D')).mean())
    rt_v2_1day_ave = {k: daily_average(v) for (k, v) in rt.items()}
    
    for b in basins:
        rt_v2_1day_ave[b].index.name = 'time'
        rt_v2_1day_ave[b] = rt_v2_1day_ave[b].dropna()
        
    with open(data_v2_dir+'nwm_rt_v2_1d.p', 'wb') as f:
        pickle.dump(rt_v2_1day_ave, f)
else:
    print('Load data averaged by day')
    with open(data_v2_dir+'nwm_rt_v2_1d.p', 'rb') as pf:
        rt_v2_1day_ave = pickle.load(pf)

Load data averaged by day
